<a href="https://colab.research.google.com/github/DAbbott93/Dean-Abbott--Dissertation/blob/main/model1_LSTM_WORD_EMBED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code for training LSTM model (with word embedding layer) with HopeEDI data. Includes code to preprocess the data for input into the LSTM model.

First we must install dependencies:

In [ ]:
#installing torch, torchvision and torchtext
!pip install torch==1.6.0 torchvision==0.7.0 torchtext==0.7.0

next we mount drive for saving files to

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Next we install the packages required for training the LSTM model:

In [ ]:
import torch
import torch.nn as nn
from torchtext import data
import torch.optim as optim
import random
import csv
from nltk import tokenize
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, multilabel_confusion_matrix, f1_score, accuracy_score
import re
import numpy as np
import matplotlib.pyplot as plt

Ensuring reproducibilty and than GPU is being used-essential as code will not run on CPU.

In [ ]:
#This is to ensure we can reproduce this result
SEED = 12345
# check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Torch
torch.manual_seed(SEED)
#Cuda algorithms
torch.backends.cudnn.deterministic = True

defining some hyperparameter choices for our model:

In [ ]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)
# set batch size
BATCH_SIZE = 64
EPOCHS = 9
dropout = 0.5
PLOT = True

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


The classifier class defines all the layers of the bi-directional LSTM and uses the torch embedding function:

In [ ]:
class classifier(nn.Module):
#ALL define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        # Constructor
        super().__init__()

        # torch embedding layer defined
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        # lstm layer defined
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            dropout=dropout)

        self.fc = nn.Linear(hidden_dim*2, output_dim) #This has to be *2 because it is a bidirectional RNN, the forward and backward components have to be concatenated together

        # activation function is Sigmoid
        self.act = nn.Sigmoid()

    def forward(self, text, text_lengths):
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]

        # pack_padded_sequence so that padded items in the sequence aren't passed to LSTM
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True)

        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #outputs are defines as:
        # hidden = [batch size, num layers * num directions,hid dim]
        # cell = [batch size, num layers * num directions,hid dim]

        # concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)

        # hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden) 

        # Final activation function called
        outputs = self.act(dense_outputs)

        return outputs


In [ ]:
# define metric, called in training function to monitor perfromance
def binary_accuracy(preds, y):
    # round predictions to the closest integer for comparison
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train(model, iterator, optimizer, train_loss_vector, train_acc_vector, criterion):
    # initialize every epoch with 0 loss and accuracy 
    epoch_loss = 0
    epoch_acc = 0

    # set the model to training phase
    model.train()

    for batch in iterator:
        # resets the gradients after every batch so that they are not being accumulated
        optimizer.zero_grad()

        # retrieve text and no. of words
        text, text_lengths = batch.text

        # convert to 1D tensor for comparion with true labels
        predictions = model(text, text_lengths).squeeze()

        # compute the loss between predictions and true labels
        loss = criterion(predictions, batch.label.type(torch.float32))

        # binary accuracy is calculated
        acc = binary_accuracy(predictions, batch.label)

        # backpropage the loss and calculate the gradients
        loss.backward()

        # update the weights with respect to the loss
        optimizer.step()

        # loss and accuracy for each batch added together, note zeroed for every new epoch
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    train_loss_vector.append(epoch_loss)
    train_acc_vector.append(epoch_acc)
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
#f1 loss defined for evaluation procedure later
def f1_loss(y_pred:torch.Tensor, y_true:torch.Tensor, is_training=False):
    
   
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)   
    y_pred = torch.round(torch.sigmoid(y_pred))
   
    #true positives, true negatives, false positives and false negatives
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7 
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1, precision, recall, tp, tn, fp, fn


In [ ]:
#evaluation function defined for training procedure
def evaluate(epoch, model, iterator, val_loss_vector, val_accuracy_vector, criterion):
    # initialize every epoch
    val_epoch_loss = 0
    val_epoch_acc = 0
    # deactivating dropout layers
    model.eval()
    # deactivates autograd for evalutaing purposes
    with torch.no_grad():
        for batch in iterator:
            # retrieve text and no. of words
            text, text_lengths = batch.text

            # convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()

            # compute loss and accuracy
            loss = criterion(predictions, batch.label.type(torch.float32))
            acc = binary_accuracy(predictions, batch.label)

            # keep track of loss and accuracy
            val_epoch_loss += loss.item()
            val_epoch_acc += acc.item()

        val_loss_vector.append(val_epoch_loss)
        val_accuracy_vector.append(val_epoch_acc)
    return val_epoch_loss / len(iterator), val_epoch_acc / len(iterator)


In [ ]:
!pip install datasets
from datasets import load_dataset
from pandas import DataFrame

#Hope dataset is called 'dataset', it contains a training and validation set
dataset= load_dataset("hope_edi", "english")

#training dataset is separated from the dataset and called df_train
#we replace positive and negative labels with  0 and 1 respectively
#this is then saved as a tsv file on google drive 
df_train=DataFrame({'text':dataset['train']['text'], 'label': dataset['train']['label']})
print(df_train.shape)
print(df_train['label'].value_counts())
df_train['label'] = df_train['label'].replace([0], "positive")
df_train['label'] = df_train['label'].replace([1], "negative")
df_train.to_csv('/content/drive/MyDrive/Hope_Dataset/train.tsv', sep="\t",index=False)


#Validation dataset is separted from dataset and called df_val
#we replace positive and negative labels with  0 and 1 respectively
#this is then saved as a tsv file on google drive 
df_val=DataFrame({'text':dataset['validation']['text'], 'label': dataset['validation']['label']}) 
df_val['label'] = df_train['label'].replace([0], "positive")
df_val['label'] = df_train['label'].replace([1], "negative")
df_val.to_csv('/content/drive/MyDrive/Hope_Dataset/test.tsv', sep="\t", index=False)

df_val.head()
df_train.tail()

#we remove a third label '2' which is the non english category
df_train=df_train.drop(df_train.query('label == 2').sample(frac=1).index)
df_val=df_val.drop(df_val.query('label == 2').sample(frac=1).index)
  

Reusing dataset hope_edi (/root/.cache/huggingface/datasets/hope_edi/english/1.0.0/fff5cf6e767fe3d1de7c5df863565bdce10bfe79dfb0b2ce42d320c3864497e3)


(22762, 2)
1    20778
0     1962
2       22
Name: label, dtype: int64


In [ ]:
df_val['label'].value_counts()

negative    2608
positive     232
Name: label, dtype: int64

In [ ]:
fields = [('text', TEXT), ('label', LABEL)]
#loading custom dataset
training_data=data.TabularDataset(path ="/content/drive/MyDrive/Hope_Dataset/train.tsv",format = 'tsv',fields = fields,skip_header = True)
##fields = [('text', TEXT), ('label', LABEL)]
# loading custom dataset


train_data, valid_data = training_data.split(split_ratio=0.8, random_state = random.seed(SEED))   #, random_state = random.seed(SEED)
# train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED))



print(len(train_data))
print(len(valid_data))
#print(len(test_data))
# initialize glove embeddings
TEXT.build_vocab(train_data, min_freq=4, vectors="glove.6B.100d")   #this determines the size of the embedding vector
LABEL.build_vocab(train_data)

# No. of unique tokens in text
print("Size of TEXT vocabulary:", len(TEXT.vocab))

# No. of unique tokens in label
print("Size of LABEL vocabulary:", len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))

# Word dictionary
print(TEXT.vocab.stoi)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


18210
4552
Size of TEXT vocabulary: 5453
Size of LABEL vocabulary: 3
[('.', 12525), ('the', 8928), ('to', 6559), ('I', 6232), ('a', 5920), ('is', 5769), ('and', 5261), ('that', 4352), ('of', 4213), ('you', 4054)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fa16121aa10>>, {'<unk>': 0, '<pad>': 1, '.': 2, 'the': 3, 'to': 4, 'I': 5, 'a': 6, 'is': 7, 'and': 8, 'that': 9, 'of': 10, 'you': 11, '!': 12, 'it': 13, '?': 14, 'in': 15, 'are': 16, 'do': 17, 'people': 18, 'lives': 19, 'matter': 20, 'not': 21, "'s": 22, "n't": 23, 'for': 24, 'all': 25, ' ': 26, 'this': 27, 'they': 28, 'was': 29, 'with': 30, 'be': 31, 'black': 32, 'have': 33, 'so': 34, 'just': 35, 'like': 36, 'what': 37, 'on': 38, 'about': 39, '...': 40, 'but': 41, 'we': 42, 'as': 43, 'my': 44, 'she': 45, 'The': 46, 'who': 47, 'n’t': 48, 'your': 49, '’s': 50, 'It': 51, 'i': 52, 'white': 53, 'if': 54, 'or': 55, 'because': 56, 'say': 57, 'me': 58, 'think': 59, 'he': 60, 'her': 61, 'one': 62

In [ ]:
# check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load an iterator
#train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data),batch_size=BATCH_SIZE, sort_key=lambda x: len(x.text),sort_within_batch=True,device=device)
train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data),batch_size=BATCH_SIZE, sort_key=lambda x: len(x.text),sort_within_batch=True,device=device)


# define hyperparameters
size_of_vocab = len(TEXT.vocab)
print(size_of_vocab)
embedding_dim = 100
num_hidden_nodes = 128
num_output_nodes = 1
num_layers = 2
bidirection = True
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
  #
# # instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers,
                      bidirectional=True, dropout=dropout, pad_idx=PAD_IDX)
print(model)


# No. of trianable parameters
print(f'The model has {count_parameters(model):,} trainable parameters')

cuda
5453
classifier(
  (embedding): Embedding(5453, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 1,176,341 trainable parameters


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:

# Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors #replaces the initial weights of embedding later with the pre-trained embeddings
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

print(model.embedding.weight.data)

torch.Size([5453, 100])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.0587,  0.0901, -0.0142,  ..., -0.1028,  0.0044, -0.2619],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [ ]:
# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# push to cuda if available

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model.cuda()
CUDA_LAUNCH_BLOCKING=1

print('cuda parmaters:', next(model.parameters()).is_cuda)
criterion = criterion.to(device)


best_valid_loss = float('inf')
val_loss_vector = []
val_acc_vector = []
train_loss_vector = []
train_acc_vector = []

device: cuda
cuda parmaters: True


In [ ]:

for epoch in range(20):
    print("Epoch: ", epoch)

    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, train_loss_vector, train_acc_vector, criterion)

    # evaluate the model
    valid_loss, valid_acc = evaluate(epoch, model, valid_iterator, val_loss_vector, val_acc_vector, criterion)
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        #torch.save(model.state_dict(), 'lstm.pt')


    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')


print(val_loss_vector)


Epoch:  0


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


	 Val. Loss: 0.230 |  Val. Acc: 91.32%
	Train Loss: 0.288 | Train Acc: 91.01%
Epoch:  1
	 Val. Loss: 0.209 |  Val. Acc: 92.23%
	Train Loss: 0.212 | Train Acc: 91.95%
Epoch:  2
	 Val. Loss: 0.208 |  Val. Acc: 92.62%
	Train Loss: 0.179 | Train Acc: 93.30%
Epoch:  3
	 Val. Loss: 0.214 |  Val. Acc: 92.34%
	Train Loss: 0.149 | Train Acc: 94.39%
Epoch:  4
	 Val. Loss: 0.245 |  Val. Acc: 92.40%
	Train Loss: 0.126 | Train Acc: 95.28%
Epoch:  5
	 Val. Loss: 0.240 |  Val. Acc: 91.97%
	Train Loss: 0.104 | Train Acc: 96.18%
Epoch:  6
	 Val. Loss: 0.249 |  Val. Acc: 92.21%
	Train Loss: 0.089 | Train Acc: 96.69%
Epoch:  7
	 Val. Loss: 0.369 |  Val. Acc: 91.12%
	Train Loss: 0.076 | Train Acc: 97.17%
Epoch:  8
	 Val. Loss: 0.375 |  Val. Acc: 92.06%
	Train Loss: 0.063 | Train Acc: 97.62%
Epoch:  9
	 Val. Loss: 0.431 |  Val. Acc: 90.69%
	Train Loss: 0.050 | Train Acc: 98.12%
Epoch:  10
	 Val. Loss: 0.571 |  Val. Acc: 90.06%
	Train Loss: 0.044 | Train Acc: 98.17%
Epoch:  11
	 Val. Loss: 0.569 |  Val. Acc

In [ ]:
def f1_loss(y_pred:torch.Tensor, y_true:torch.Tensor, is_training=False):
    '''Calculate F1 score. Can work with gpu tensors'''
   
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
    
    y_pred = torch.round(torch.sigmoid(y_pred))
   
   

    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1, precision, recall, tp, tn, fp, fn


In [ ]:
unseen_data_2 = data.TabularDataset(path="/content/drive/MyDrive/Hope_Dataset/test.tsv",format='tsv', fields= [('text', TEXT), ('label', LABEL)], skip_header=True)

unseen_train_data, unseen_data = unseen_data_2.split(split_ratio=0.99,
                                                      random_state=random.seed(
                                                          SEED)) 

print(len(unseen_train_data))
print(len(unseen_data))

LABEL.build_vocab(unseen_train_data)
unseen_train_data_iter, unseen_data_iter = data.BucketIterator.splits((unseen_train_data, unseen_data),
                                                                        batch_size=256,
                                                                        sort_key=lambda x: len(x.text),
                                                                        sort_within_batch=True, device=device)



/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


2815
28


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
def evaluate_testset(model, unseen_train_data_iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    tp_total = 0
    tn_total = 0 
    fp_total = 0
    fn_total =0
    total_no_inputs = 0
    model.eval()
    
    with torch.no_grad():
    
        for batch in unseen_train_data_iter:
            #print('batch:', batch)
            total_no_inputs += 256
            #print(total_no_inputs)

            text, text_lengths = batch.text

        
            predictions = model(text, text_lengths).squeeze()
            #predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            #print('label:', batch.label)
            #print('preds;', predictions)
            acc = binary_accuracy(predictions, batch.label)
            f1, precision, recall, tp, tn, fp, fn= f1_loss(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            tp_total +=tp.item()
            fp_total +=fp.item()
            tn_total +=tn.item()
            fn_total +=fn.item()
           

    return epoch_loss / len(unseen_train_data_iter), epoch_acc / len(unseen_train_data_iter), tp_total, tn_total, fp_total, fn_total 



In [ ]:

unseen_test_loss, unseen_test_acc, tp_total, tn_total, fp_total, fn_total = evaluate_testset(model, unseen_train_data_iter, criterion)



/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
print(f'Unseen Test Loss: {unseen_test_loss:.3f} | Unseen Test Acc: {unseen_test_acc * 100:.2f}%')
print(  'tp:', tp_total, 'tn:', tn_total, 'fp:', fp_total, 'fn:', fn_total)
prec = tp_total/(tp_total + fp_total)
recall = tp_total/(tp_total + fn_total)
print('Recall', recall)
print('Prec', prec )

print("finished")   

Unseen Test Loss: 1.589 | Unseen Test Acc: 85.22%
tp: 142.0 tn: 966.0 fp: 1612.0 fn: 95.0
Recall 0.5991561181434599
Prec 0.08095781071835804
finished


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/Tweets.csv",encoding='latin1')
df2.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
#Training datset

df2["airline_sentiment"] = df2["airline_sentiment"].replace("neutral", "negative")
df2=df2[["text","airline_sentiment"]]
df2.columns = ['text', 'label']
df2.sample(10)


,text,label
4249,@united giving up on your direct flight from I...,negative
13429,@AmericanAir tisk tisk. Rude flight attendants...,negative
3675,"@united, I'm still frustrated I gave up my sea...",negative
4396,You have to follow me back so that I can DM @S...,negative
10081,@USAirways flight 1735 sitting fully loaded fo...,negative
5081,@SouthwestAir yall have me sleeping in the air...,negative
3604,@united thanks,positive
2950,@united can't wait!!! 787!!! @tpallini http://...,positive
14607,@AmericanAir i need someone to help me out,negative
10531,@USAirways now what? You Cancelled Flightled m...,negative


In [ ]:
df2.to_csv('/content/drive/MyDrive/AirlineTweet.tsv', sep="\t",index=False)

In [ ]:
unseen_data2 = data.TabularDataset(path="/content/drive/MyDrive/AirlineTweet.tsv",format='tsv', fields = [('text', TEXT), ('label', LABEL)], skip_header=True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [ ]:
print(df2['label'].value_counts())

negative    12277
positive     2363
Name: label, dtype: int64


In [ ]:

unseen_train_data, unseen_data = unseen_data2.split(split_ratio=0.99,
                                                      random_state=random.seed(
                                                          SEED)) 

print(len(unseen_train_data))
print(len(unseen_data))

LABEL.build_vocab(unseen_train_data)
unseen_train_data_iter, unseen_data_iter = data.BucketIterator.splits((unseen_train_data, unseen_data),
                                                                        batch_size=256,
                                                                        sort_key=lambda x: len(x.text),
                                                                        sort_within_batch=True, device=device)
                                                                  

unseen_test_loss, unseen_test_acc, tp_total, tn_total, fp_total, fn_total = evaluate_testset(model, unseen_train_data_iter, criterion)
print(f'Unseen Test Loss: {unseen_test_loss:.3f} | Unseen Test Acc: {unseen_test_acc * 100:.2f}%')
print(  'tp:', tp_total, 'tn:', tn_total, 'fp:', fp_total, 'fn:', fn_total)
prec = tp_total/(tp_total + fp_total)
recall = tp_total/(tp_total + fn_total)
print('Recall', recall)
print('Prec', prec )
F1 = 2*prec*recall/ (prec+recall)
print('F1:', F1)
print("finished")   

14494
146


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Unseen Test Loss: 1.888 | Unseen Test Acc: 81.90%
tp: 1967.0 tn: 1911.0 fp: 10236.0 fn: 380.0
Recall 0.838091180230081
Prec 0.16118987134311236
F1: 0.27037800687285224
finished


In [ ]:
df3 = pd.read_csv("/content/drive/MyDrive/coviddataset.csv",encoding='latin1')
df3.dropna(subset = ["Sentiment"], inplace=True)
df3.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16/03/2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16/03/2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16/03/2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16/03/2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16/03/2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
df3['Sentiment'] = df3['Sentiment'].map({'Positive':'negative', 'Extremely Positive':"positive",'Negative':"negative",'Extremely Negative':"negative",'Neutral':"negative"})
df3=df3[["OriginalTweet","Sentiment"]]
df3.columns = ['text', 'label']
df3.sample(10)


,text,label
2439,"Any news on mortgage companies, car finance or...",negative
3947,Music to listen to on your next trip to the gr...,negative
1643,At the local grocery store now and almost all ...,negative
7254,Experts can discuss how #coronavirus is likely...,negative
1707,(TAKE3) High time PM @ScottMorrisonMP sat down...,negative
510,"We have enough food, assures PM\n\nhttps://t.c...",negative
7349,Please be smart and prepare. Here my take Sto...,positive
2652,IKEA closing all U S stores due to COVID 19 wi...,negative
6987,People be embarrassed to wear masks glasses am...,negative
6408,@ThabitSenior @mpoki_m @msangijeff @danielmarr...,negative


In [ ]:
df3.to_csv('/content/drive/MyDrive/covid_test.tsv', sep="\t",index=False)

In [ ]:
unseen_data3 = data.TabularDataset(path="/content/drive/MyDrive/covid_test.tsv",format='tsv', fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [ ]:
print(df3['label'].value_counts())

negative    6842
positive    1212
Name: label, dtype: int64


In [ ]:
unseen_train_data, unseen_data = unseen_data3.split(split_ratio=0.99,
                                                      random_state=random.seed(
                                                          SEED)) 


print(len(unseen_train_data))
print(len(unseen_data))

LABEL.build_vocab(unseen_train_data)
unseen_train_data_iter, unseen_data_iter = data.BucketIterator.splits((unseen_train_data, unseen_data),
                                                                        batch_size=256,
                                                                        sort_key=lambda x: len(x.text),
                                                                        sort_within_batch=True, device=device)
                                                                  


unseen_test_loss, unseen_test_acc, tp_total, tn_total, fp_total, fn_total = evaluate_testset(model, unseen_train_data_iter, criterion)
print(f'Unseen Test Loss: {unseen_test_loss:.3f} | Unseen Test Acc: {unseen_test_acc * 100:.2f}%')
print(  'tp:', tp_total, 'tn:', tn_total, 'fp:', fp_total, 'fn:', fn_total)
prec = tp_total/(tp_total + fp_total)
recall = tp_total/(tp_total + fn_total)
print('Recall', recall)
print('Prec', prec )
F1 = 2*prec*recall/ (prec+recall)
print('F1:', F1)
print("finished")   

7973
81


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Unseen Test Loss: 1.562 | Unseen Test Acc: 82.70%
tp: 1092.0 tn: 1063.0 fp: 5705.0 fn: 113.0
Recall 0.9062240663900415
Prec 0.16065911431513905
F1: 0.2729317670582354
finished
